In [1]:
import refinitiv.data as rd
import numpy as np
import pandas as pd
import matplotlib as plt
import warnings
import plotly.graph_objects as go
import time
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x7f7c52145730 {name='codebook'}>

In [3]:
# which index is used as a basis of comparison
index = '.SPX'
# what is the time range for the backtesting 
start_year = 2014
end_year = 2023
# what is the company's  minimum ESG Score (0-100) for the high ESG subset
ESG_score_threshold_upper = 70

ESG_score_threshold_lower = 30

In [4]:
sp = rd.get_data(
    index, 
    fields=['TR.PriceClose.date', 'TR.PriceClose'],
    parameters={
        'SDate': '{}-01-01'.format(start_year),
        'EDate': '{}-01-01'.format(end_year),
        'Frq': 'CQ'  # Daily frequency
    }
)

sp['Rebased'] = sp['Price Close'] * 100 / sp['Price Close'][0]
sp.head()

,Instrument,Date,Price Close,Rebased
0,.SPX,2014-03-31,1872.34,100.0
1,.SPX,2014-06-30,1960.23,104.694126
2,.SPX,2014-09-30,1972.29,105.33824
3,.SPX,2014-12-31,2058.9,109.964002
4,.SPX,2015-03-31,2067.89,110.44415


In [5]:
def getSubset(dFrame, low_high):
    # filter out the companies with low ESG score at the begining year
    if low_high == "low":
        dFrame = dFrame.loc[
            (dFrame['ESG Combined Score'] < ESG_score_threshold_lower) 
        ]
    elif low_high == "high":
        dFrame = dFrame.loc[
            (dFrame['ESG Combined Score'] > ESG_score_threshold_upper)
        ]
    return dFrame['Instrument'].tolist()

In [8]:
def getDataForYear(year, low_high):
    print(f'Getting data for: {year}')
    # get index constituents at the begining year
    df = rd.get_data('{}{}({}-01-01)'.format('0#', index, year), fields=['TR.TRESGCScore'], parameters={'SDate': '{}-01-01'.format(year), 'Period': 'FY0'})
    # filter out the instruments based on ESG ratings
    subset = getSubset(df, low_high)
    # get the performance data for this subset
    df2 = rd.get_data(subset, fields=['TR.PriceClose.date', 'TR.PriceClose'], parameters={'SDate': '{}-01-01'.format(year), 'EDate': '{}-01-01'.format(year + 1), 'Frq':'CQ'})
    df2 = df2.dropna()
    # consolidate the price data for instruments
    alphaList = []
    ser = df2['Date'].value_counts()
    for x in ser[ser > 5].index:
        ser = df2.loc[df2['Date'] == x].sum(numeric_only=True)
        alphaList.append({'Date': x, 'Price Close': ser['Price Close']})
    return pd.DataFrame(alphaList)


In [9]:
df_high = pd.DataFrame()
for x in range(start_year, end_year):
    ret = getDataForYear(x, "high")
    df_high = pd.concat([df_high, ret])
    time.sleep(3)
df_high = df_high.sort_values(by=['Date'])
df_high = df_high.reset_index(drop=True)
df_high['Rebased'] = df_high['Price Close'] * 100 / df_high['Price Close'][0]
df_high.head()
df_high.to_csv('Combined_ESG_Score_high70.csv', index=False)

Getting data for: 2014
Getting data for: 2015
Getting data for: 2016
Getting data for: 2017
Getting data for: 2018
Getting data for: 2019
Getting data for: 2020
Getting data for: 2021
Getting data for: 2022


In [10]:
df_low = pd.DataFrame()
for x in range(start_year, end_year):
    ret = getDataForYear(x, "low")
    df_low = pd.concat([df_low, ret])
    time.sleep(3)
df_low = df_low.sort_values(by=['Date'])
df_low = df_low.reset_index(drop=True)
df_low['Rebased'] = df_low['Price Close'] * 100 / df_low['Price Close'][0]
df_low.head()
df_low.to_csv('Combined_ESG_Score_low30.csv', index=False)

Getting data for: 2014
Getting data for: 2015
Getting data for: 2016
Getting data for: 2017
Getting data for: 2018
Getting data for: 2019
Getting data for: 2020
Getting data for: 2021
Getting data for: 2022


In [ ]:
# plot the graphs together
res = [go.Scatter(x=df_high['Date'], y=df_high['Rebased'], name=f'ESG Combined Score > {ESG_score_threshold}%', mode='lines'),
        go.Scatter(x=sp['Date'], y=sp['Rebased'], name='S&P 500', mode='lines')]

lay1 = {'shapes': [
        {'type': 'line','x0': '{}-01-01'.format(start_year),'y0': 100, 'x1':'{}-01-01'.format(end_year),'y1': 100,'line': {'width': 1, 'color': 'white'},}
    ],
    'showlegend': True
}

lay2 = {
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'plot_bgcolor': 'rgba(0,0,0,0)'
}

lay3 = {
    'title': f'Relative perfomance of high ESG score vs Index, for the last {end_year - start_year} years',
    'font': {
        'size': 14,
        'color': 'white'
    }
}

fig = go.Figure(data=res)
fig.update_xaxes(showgrid=False, gridwidth=1, gridcolor='darkgrey')
fig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='darkgrey')
fig.update_layout(lay1)
fig.update_layout(lay2)
fig.update_layout(lay3)
fig.show()